In [17]:
from google.colab import drive
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.callbacks import EarlyStopping

drive.mount('/content/drive')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [23]:

file_path = '/content/drive/MyDrive/bitcoin/BTC-USD.csv'


df = pd.read_csv(file_path)

total_rows = len(df)
train_rows = int(0.7 * total_rows)


train_df = df.iloc[:train_rows]
val_df = df.iloc[train_rows:]

train_df.to_csv('training_data.csv', index=False)
val_df.to_csv('validation_data.csv', index=False)


In [24]:
print(df)
print(train_df)
print(val_df)

            Date          Open          High           Low         Close  \
0     2014-09-17    465.864014    468.174011    452.421997    457.334015   
1     2014-09-18    456.859985    456.859985    413.104004    424.440002   
2     2014-09-19    424.102997    427.834991    384.532013    394.795990   
3     2014-09-20    394.673004    423.295990    389.882996    408.903992   
4     2014-09-21    408.084991    412.425995    393.181000    398.821014   
...          ...           ...           ...           ...           ...   
3515  2024-05-02  58253.703125  59602.296875  56937.203125  59123.433594   
3516  2024-05-03  59122.300781  63320.503906  58848.312500  62889.835938   
3517  2024-05-04  62891.031250  64494.957031  62599.351563  63891.472656   
3518  2024-05-05  63892.453125  64610.890625  62955.304688  64031.132813   
3519  2024-05-06  64032.632813  65425.679688  63750.796875  63750.796875   

         Adj Close       Volume  
0       457.334015     21056800  
1       424.440002 

In [25]:

df['Date'] = pd.to_datetime(df['Date'])

df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day


df.drop(columns=['Date'], inplace=True)

X = df[['Year','Month','Day','Open', 'Low','Close','Adj Close', 'Volume']]
y = df['High']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(1)
])

model.compile(optimizer='adam', loss='mean_squared_error')

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history = model.fit(X_train_scaled, y_train, epochs=100, batch_size=32, validation_split=0.2, callbacks=[early_stopping])
train_loss = model.evaluate(X_train_scaled, y_train)
test_loss = model.evaluate(X_test_scaled, y_test)

print("Train Loss:", train_loss)
print("Test Loss:", test_loss)


Epoch 1/100
71/71 [==============================] - 2s 8ms/step - loss: 609442048.0000 - val_loss: 582841600.0000
Epoch 2/100
71/71 [==============================] - 0s 5ms/step - loss: 427930272.0000 - val_loss: 150464448.0000
Epoch 3/100
71/71 [==============================] - 0s 4ms/step - loss: 116407288.0000 - val_loss: 76509696.0000
Epoch 4/100
71/71 [==============================] - 0s 5ms/step - loss: 54102300.0000 - val_loss: 28368328.0000
Epoch 5/100
71/71 [==============================] - 0s 4ms/step - loss: 17254348.0000 - val_loss: 6950484.0000
Epoch 6/100
71/71 [==============================] - 0s 5ms/step - loss: 4518367.5000 - val_loss: 2086833.2500
Epoch 7/100
71/71 [==============================] - 0s 5ms/step - loss: 1882723.6250 - val_loss: 1258319.1250
Epoch 8/100
71/71 [==============================] - 0s 5ms/step - loss: 1352150.5000 - val_loss: 1031727.4375
Epoch 9/100
71/71 [==============================] - 0s 4ms/step - loss: 1102959.7500 - val_loss: 